In [ ]:
import pymysql

ENDPOINT = '10.10.211.59'
PORT = 3306
USERNAME = 'read_only'
DBNAME = 'CAPC_ACCURACY_REPORT'
PASSWORD = 'P@ssw0rd@123' # Change this based on your own password
CURSORCLASS = pymysql.cursors.DictCursor

In [ ]:
def initiate_local_connection():
    try:
        connection = pymysql.connect(host=ENDPOINT,
                                     port=PORT,
                                     user=USERNAME,
                                     passwd=PASSWORD,
                                     db=DBNAME,
                                     cursorclass=CURSORCLASS)
        print('[+] Local Connection Successful')
    except Exception as e:
        print(f'[+] Local Connection Failed: {e}')
        connection = None

    return connection

In [ ]:
#  Call function and return the connection object
connection = initiate_local_connection()

In [ ]:
query = '''select * from overall_data where month in ('2023-07') and client_code='CHS';'''

In [ ]:
df_query = pd.read_sql(query, con=connection)

In [ ]:
df_query

In [ ]:
df_query.to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_Overall_sheet_till_11th.xlsx')

In [ ]:
df = df_query.copy()

In [ ]:
from collections import OrderedDict
import pandas as pd
def get_accuracy(data_df, codetype):
    
    # display(data_df)
    # print(data_df.columns)
    
    accept = data_df[codetype+'_accepted_count'].sum()
    reject = data_df[codetype+'_rejected_count'].sum()
    add = data_df[codetype+'_added_count'].sum()
    
    d = OrderedDict()
    # d['facility_name'] = data_df['facility_name'].iloc[0]
    d['encounters'] = data_df.shape[0]
    
    d[codetype+'_accepted'] = accept
    d[codetype+'_rejected'] = reject
    d[codetype+'_added'] = add
    
    d[codetype+'_billed'] = accept+add
    
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [ ]:
df.groupby(['facility_code']).apply(get_accuracy, codetype='eandm').sort_values(['facility_code'],ascending=[True])

In [ ]:
df.enc_doc_id = df.enc_doc_id.astype('str')

In [ ]:
df.shape

In [ ]:
df_single_docs = df[~df.enc_doc_id.str.contains(',')]

In [ ]:
df[df.enc_visit_type.str.contains('RECHECK')][cols_to_use]

In [ ]:
df_single_docs.shape

In [ ]:
df_9954 = df[df['facility_code'] == '9954']

In [ ]:
df_9954.shape

In [ ]:
df_9954.groupby(['enc_visit_type']).apply(get_accuracy, codetype='eandm').sort_values(['eandm_rejected'],ascending=[False])

In [ ]:
df_9954.enc_visit_type = df_9954.enc_visit_type.astype('str')

In [ ]:
df.columns

In [ ]:
cols_to_use = ['encounter_id', 'service_date', 'enc_visit_type', 'enc_place_of_service', 'facility_code', 'account_number', 'eandm_suggested', 'eandm_final', 'cpt_suggested', 'cpt_final']

In [ ]:
df_9954[df_9954.enc_visit_type.str.contains('N/A')][cols_to_use]

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_Overall_sheet_till_11th.xlsx', index_col=False)

In [ ]:
df.facility_code = df.facility_code.astype('str')

In [ ]:
df_1575 = df[df['facility_code'] == '1575']

In [ ]:
df_1575[cols_to_use]

In [ ]:
cs_1575 = pd.crosstab(df_1575['clean_final'], df_1575['clean_suggested'], margins=True)

In [ ]:
import re
for i,r in df_1575.iterrows():
    df_1575.loc[i,'clean_suggested'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_suggested']))])
    df_1575.loc[i,'clean_final'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_final']))])

In [ ]:
cs_1575.to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_ALL_TARS/1902.xlsx')

In [ ]:
# df_1570[(df_1570.clean_final == '99213') & (df_1570.clean_suggested == '99212')][cols_to_use].to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_ALL_TARS/docs_1570_JULY/Output/TEST.xlsx')

In [ ]:
df_1575[(df_1575.clean_final == '99203') & (df_1575.clean_suggested == '99202')][cols_to_use]

In [ ]:
df_1575.cpt_final = df_1575.cpt_final.astype('str')

In [ ]:
df_1575[df_1575.cpt_final.str.contains('73630')][cols_to_use].eandm_final.value_counts()

In [ ]:
df_1575[~df_1575.clean_suggested.str.contains('9') & ((df_1575.clean_final.str.contains('9')))][cols_to_use]

In [ ]:
# .to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/CHS_JULY_ALL_TARS/docs_1570_JULY/Output/TEST.xlsx')

In [ ]:
cols_to_use = ['encounter_id', 'service_date', 'enc_visit_type', 'enc_place_of_service', 'facility_code', 'account_number', 'cpt_final', 'cpt_suggested','eandm_final', 'eandm_suggested','local_suggested_ruleEngine','eandm_accepted_count',
       'eandm_rejected_count', 'eandm_added_count', 'local_suggested_14_july', 'new', '17_july_head', 'New_4']

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy = df_copy[cols_to_use]

In [ ]:
df_copy.local_suggested_ruleEngine = df_copy.local_suggested_ruleEngine.astype('str')

In [ ]:
df_copy = df_copy[~df_copy.local_suggested_ruleEngine.str.contains('vlookup_null')]

In [ ]:
# df_copy.to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/local_JULY.xlsx')

In [ ]:
import re
for i,r in df_copy.iterrows():
    df_copy.loc[i,'clean_suggested'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_suggested']))])
    df_copy.loc[i,'clean_final'] = ', '.join([str(elem) for elem in re.findall(r"\b\d{5}\b", str(r['eandm_final']))])

In [ ]:
df_copy

In [ ]:
df_copy.clean_final.value_counts()

In [ ]:
df_copy.clean_suggested.value_counts()

In [ ]:
def get_counts(x):
    
    suggested = str(x['New_4'])
    final = str(x['clean_final'])
    
    if suggested != '':
        suggested_set = set(suggested.split(', '))
        final_set = set(final.split(', '))
        if '9' not in final:
            final_set.clear()    
        if '9' not in suggested:
            suggested_set.clear()
            
        accepted = len(suggested_set.intersection(final_set))
        rejected = len(suggested_set - final_set)
        added = len(final_set - suggested_set)
        
        if (rejected >= 1 or added >= 1):
            print('rejected',rejected)
            print('added',added)
            print('suggested_set',suggested_set,'\t final_set',final_set)
        return pd.Series({'accepted':accepted, 'rejected':rejected, 'added':added})

In [ ]:
count_df =  df_copy.apply(get_counts, axis=1)

In [ ]:
df2 = pd.concat([df_copy, count_df], axis = 1)

In [ ]:
df2

In [ ]:
df2.eandm_rejected_count.sum()

In [ ]:
df2.rejected.sum()

In [ ]:
from collections import OrderedDict
def get_accuracy_new(data_df):
    
    # display(data_df)
    # print(data_df.columns)
    
    accept = data_df['accepted'].sum()
    reject = data_df['rejected'].sum()
    add = data_df['added'].sum()
    
    d = OrderedDict()
    d['encounters'] = data_df.shape[0]
    
    d['New'+'_accepted'] = accept
    d['New'+'_rejected'] = reject
    d['New'+'_added'] = add
    
    d['New'+'_billed'] = accept+add
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [ ]:
df2.groupby(['facility_code']).apply(get_accuracy_new).sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/test_local.xlsx')

In [ ]:
df2.groupby(['facility_code']).apply(get_accuracy, codetype='eandm').sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/test_with_rule.xlsx')

In [ ]:
new_df = df2.copy()

In [ ]:

new_df.info()


In [ ]:
for i, r in new_df.iterrows():
    if '99024' in str(r['cpt_suggested']) or '0502F' in str(r['cpt_suggested']):
        new_df.at[i, 'accepted'] = 1

In [ ]:
new_df

In [ ]:
new_df.groupby(['facility_code']).apply(get_accuracy_new).sort_values(['facility_code'],ascending=[True]).to_excel('/home/trishalaswain/ClientData/CHS/CHS_JULY/after_applying_0502F_and_99024_rule.xlsx')

In [ ]:
new_df.to_excel('')